# Лабораторная работа по теме "Metamorphic testing"

Ссылка на материалы занятия https://drive.google.com/drive/folders/1dbq1scqU22XPFCrVmz75gpRsb49QjtcM?usp=drive_link

Литература

1) Barr E.T., Harman M., McMinn P., Shahbaz M., Yoo S. The oracle problem in software testing: A survey. IEEE transactions on software engineering. 41(5). 2014. P. 507-525. https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=6963470

2) Tsong Yueh Chen, Fei-Ching Kuo, Huai Liu, Pak-Lok Poon, Dave Towey, T. H.
Tse, and Zhi Quan Zhou. 2018. Metamorphic Testing: A Review of Challenges
and Opportunities. 51, 1, Article 4 (Jan. 2018), 27 pages. https://doi.org/10.1145/3143561

3) M. Srinivasan, M. P. Shahri, I. Kahanda and U. Kanewala, "Quality Assurance of Bioinformatics Software: A Case Study of Testing a Biomedical Text Processing Tool Using Metamorphic Testing", 2018 IEEE/ACM 3rd International Workshop on Metamorphic Testing (MET), Gothenburg, 2018, pp. 26-33. https://arxiv.org/pdf/1802.07354.pdf


**Критерии оценки.**

Условная оценка - 10 баллов за лабораторную. Для получения зачёта достаточно набрать 6.



- Задание 1. (4 вопроса, 1 балл)
- Задание 2. (9 баллов)

## Задание 1. Теоретическая часть

Ответьте на следующие вопросы (*1 балл*).
1. Опишите суть проблемы тестового оракула.
2. Приведите примеры задач, для которых обычное тестирование с оракулом не подходит.
3. Перечислите методы, которые применяются для решения этой проблемы.
4. Дайте определение тестового инварианта (metamorphic relation).





Приведите свои ответы здесь:

1. Чтобы првоерить ответы программы, нужно чтобы ответ уже был, либо легко проверяем.
2. Например, задачи, связанные с весами нейронной сети. Задачи по генерации текста.
3. Implicit oracles, comparison testing, fuzzing, metamorphic testing, assertion testing.
4. Вместо проверки
правильности каждого конкретного ответа программы проверяется
выполнение тестового инварианта (metamorphic relation), вычисляемого
для нескольких наборов исходных данных и соответствующих им ответов
программы.

## Задание 2. Разработка тестовых инвариантов

Дана модель для распознавания сущностей в тексте.
- Придумайте и реализуйте 2 теста с тестовым оракулом (обычные тесты с правильными ответами) (*1 балл*)
- Придумайте и реализуйте не менее 3 тестовых инвариантов (metamorphic relations) для её проверки - (*суммарно 6 баллов, теоретическое описание - по 1 баллу, реализация - по 1 баллу*)
- Сравните полученные тесты. В чем преимущества каждого из методов? В чём недостатки? (*2 балла*)

*Указание*. Можете воспользоваться идеями со слайда "Bio-entity recognition" или из статьи "Quality Assurance of Bioinformatics Software: A Case Study of Testing a Biomedical Text Processing Tool Using Metamorphic Testing" из списка литературы.

**Правила оформления.**

Для каждого инварианта необходимо описать
 - из каких предположений о модели он вытекает,
 - формальное описание (желательно с формулой),
 - запуск на 1-2 примерах тестовых данных.

Теоретическую часть можно оформить в ячейке markdown.

### Тестируемая система

Модель позволяет искать сущности в тексте. Ниже приведены примеры того, как можно с ней работать.

In [1]:
# ! pip install spacy==2.2.4

In [2]:
import spacy
from spacy import displacy

In [3]:
# example of model usage
def render(text):

    nlp = spacy.load('en_core_web_sm') # model

    doc = nlp(text) # data processing

    # FYI all the properties
    props = [p for p in dir(doc.ents[0]) if p[0] != '_']
    print(props)

    # custom processing of the answer

    # get counts of entities
    ent_labels = [e.label_ for e in doc.ents]
    freq = dict()
    for l in ent_labels:
        freq[l] = ent_labels.count(l)
    print(freq)

    # get coordinates of entities
    coordinates = [e.start_char for e in doc.ents]
    print(coordinates)

    # render the answer
    displacy.render(doc, style='ent', jupyter=True)


Модель позволяет искать сущности в тексте. В ячейке ниже приведены примеры того, как можно с ней работать.

In [4]:
text = """But Google is starting from behind. The company made a late push
into hardware, and Apple’s Siri, available on iPhones, and Amazon’s Alexa
software, which runs on its Echo and Dot devices, have clear leads in
consumer adoption."""

render(text)

['as_doc', 'char_span', 'conjuncts', 'doc', 'end', 'end_char', 'ent_id', 'ent_id_', 'ents', 'get_extension', 'get_lca_matrix', 'has_extension', 'has_vector', 'id', 'id_', 'kb_id', 'kb_id_', 'label', 'label_', 'lefts', 'lemma_', 'n_lefts', 'n_rights', 'noun_chunks', 'orth_', 'remove_extension', 'rights', 'root', 'sent', 'sentiment', 'sents', 'set_extension', 'similarity', 'start', 'start_char', 'subtree', 'tensor', 'text', 'text_with_ws', 'to_array', 'vector', 'vector_norm', 'vocab']
{'ORG': 5, 'PERSON': 1, 'LOC': 1}
[4, 84, 92, 111, 124, 133, 167]



### Пример оформления инварианта

Рассмотрим задачу поиска подстроки в строке.

Предполагаем, что алгоритм должен находить все вхождения подстроки.

> **MR1.** Если в строке S найдёна некоторая подстрока s ровно k раз, то в строке SS она будет найдена не менее 2k раз (возможны нахождения на месте склейки строк).

Формально. Пусть S - строка, s - её подстрока, f(S,s) - определённое программой число вхождений s в S. Тогда f(SS,s) >= 2*f(S,s).


In [5]:
import re
import unittest

# function for testing
def func_to_test(substr, string):
    return re.finditer(pattern=substr, string=string)


class TestStringMethods(unittest.TestCase):

    def test_with_oracle1(self):
        # input data
        big_string = 'abacaba'
        substr = 'aba'

        # correct answer
        right_answer = [0, 4]

        indices = func_to_test(substr, big_string)
        answer = [index.start() for index in indices]

        self.assertTrue(answer == right_answer)

    def test_metamorphic1(self):
        # input data
        big_string1 = 'abacab'
        big_string2 = big_string1 + big_string1
        substr = 'aba'

        # first answer
        indices = func_to_test(substr, big_string1)
        indices1 = [index.start() for index in indices]

        # second answer
        indices = func_to_test(substr, big_string2)
        indices2 = [index.start() for index in indices]

        # metamorphic relation
        self.assertTrue(2*len(indices1) <= len(indices2))

Ниже мы запускаем все объявленные тестовые случаи

In [6]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK


### Реализуйте собственные тестовые инварианты ниже

## 1 Инвариант

```DATE``` не зависит от регистра. Значит, можно получить те же результаты для ```s``` как и для ```s.lower()``` и ```s.upper()```

## 2 Инвариант

Категории не зависят от порядка предложений, значит результат ```s1 + s2``` должен совпадать с ```s2 + s1```

Т. е. for i in cat: cat1[i] = cat2[i]

## 3 Инвариант

Общее оличество частот для категорий (то есть сумма по каждой категории) должно совпадать с количеством позиций в тексте.

Т. е. sum(cat[i]) == len(freq)

In [64]:
# для корректного вывода результатов
def get_result(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    props = [p for p in dir(doc.ents[0]) if p[0] != '_']
    ent_labels = [e.label_ for e in doc.ents]
    freq = dict()
    for l in ent_labels:
        freq[l] = ent_labels.count(l)
    coordinates = [e.start_char for e in doc.ents]
    return (freq, coordinates)

In [66]:
import unittest

class TestStringMethods(unittest.TestCase):

    def test_with_oracle1(self):
        test_text = """One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin.
        He lay on his armour-like back, and if he lifted his head a little he could see his brown belly, slightly domed and divided by arches into stiff sections."""

        result = get_result(test_text)
        correct_result = ({'TIME': 1, 'PERSON': 1}, [0, 18])

        self.assertTrue(result == correct_result)


    def test_with_oracle2(self):
        test_text = """The quick, brown fox jumps over a lazy dog. DJs flock by when MTV ax quiz prog. Junk MTV quiz graced by fox whelps.
        Bawds jog, flick quartz, vex nymphs. Waltz, bad nymph, for quick jigs vex! Fox nymphs grab quick-jived waltz.
        Brick quiz whangs jumpy veldt fox. Bright vixens jump; dozy fowl quack. Quick wafting zephyrs vex bold Jim.
        Quick zephyrs blow, vexing daft Jim. Sex-charged fop blew my junk TV quiz. How quickly daft jumping zebras vex.
        Two driven jocks help fax my big quiz."""

        result = get_result(test_text)
        correct_result = ({'ORG': 3, 'PERSON': 5, 'CARDINAL': 1}, [62, 85, 162, 173, 200, 262, 347, 393, 482])

        self.assertTrue(result == correct_result)


    def test_metamorphic1(self):
        test_text = """Apple Inc. is an American multinational technology company headquartered in Cupertino,
        California. As of March 2023, Apple is the world's biggest company by market capitalization, and with US$394.3 billion the largest technology company by 2022 revenue.
        As of June 2022, Apple is the fourth-largest personal computer vendor by unit sales; the largest manufacturing company by revenue;
        and the second-largest mobile phone manufacturer in the world.
        It is considered one of the Big Five American information technology companies, alongside Alphabet (parent company of Google), Amazon, Meta Platforms, and Microsoft.
        Apple was founded as Apple Computer Company on April 1, 1976, by Steve Wozniak, Steve Jobs (1955–2011) and Ronald Wayne to develop and sell Wozniak's Apple I personal computer."""


        result = get_result(test_text)[0]
        result_lower = get_result(test_text.lower())[0]
        result_upper = get_result(test_text.lower())[0]

        for test_result in [result_lower, result_upper]:
            # проверка на одновременное присутствие (или отсутствие) ключа в обоих словарях
            self.assertTrue(("DATE" in result and "DATE" in test_result) or
                            ("DATE" not in result and "DATE" not in test_result))
            if "DATE" in result:
              self.assertTrue(result["DATE"] == test_result["DATE"])


    def test_metamorphic2(self):
        test_text1 = """Apple Inc. is an American multinational technology company headquartered in Cupertino, California."""
        test_text2 = """As of March 2023, Apple is the world's biggest company by market capitalization, and with US$394.3 billion the largest technology company by 2022 revenue."""


        result1 = get_result(test_text1 + test_text2)
        result2 = get_result(test_text2 + test_text1)

        self.assertTrue(result1[0] == result2[0])


    def test_metamorphic3(self):
        test_text = """Apple Inc. is an American multinational technology company headquartered in Cupertino,
        California. As of March 2023, Apple is the world's biggest company by market capitalization, and with US$394.3 billion the largest technology company by 2022 revenue.
        As of June 2022, Apple is the fourth-largest personal computer vendor by unit sales; the largest manufacturing company by revenue;
        and the second-largest mobile phone manufacturer in the world.
        It is considered one of the Big Five American information technology companies, alongside Alphabet (parent company of Google), Amazon, Meta Platforms, and Microsoft.
        Apple was founded as Apple Computer Company on April 1, 1976, by Steve Wozniak, Steve Jobs (1955–2011) and Ronald Wayne to develop and sell Wozniak's Apple I personal computer."""

        result = get_result(test_text)
        sum = 0
        for val in result[0].values():
          sum += val

        self.assertTrue(sum == len(result[1]))

In [67]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 6.676s

OK


## Результаты

Тест с оракулом дает однозначное представление о поведении функции. Однако у него есть недостаток - нужно иметь сами правильные ответы.

Тест с инвариантом позволяет проверять различные гипотезы на больших объемах данных без реальных ответов - все зависит от гипотезы, условия, которое мы предъявляем к функции. Однако это все еще не дает безошибочных результатов (во время тестирования, столкнулся с проблемой инвариантности имен для ```s.lower()```